# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [ ]:
import pandas as pd
from pymystem3 import Mystem

In [1]:
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Название столбцов записаны в правильном виде. В столбцах 'days_employed' и 'total_income' одинаковое количество пропусков. В столбце 'days_employed' опыт в днях записан непонятным образом, изменять его не буду, для будущих вычислений он не потребуется.   В столбце 'education' записи разного регистра, изменять тоже не буду, думаю тоже не понадобятся

In [3]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


## Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
#Заполнил пропуски согласно медианной зарплаты для разных групп занятости
data_group = data.groupby('income_type')['total_income'].median()
data_group


income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [5]:
data.loc[(data['income_type'] == 'безработный') & (data['total_income'].isna()),'total_income'] = 131339.751676
data.loc[(data['income_type'] == 'в декрете') & (data['total_income'].isna()),'total_income'] = 53829.130729
data.loc[(data['income_type'] == 'госслужащий') & (data['total_income'].isna()),'total_income'] = 150447.935283
data.loc[(data['income_type'] == 'компаньон') & (data['total_income'].isna()),'total_income'] = 172357.950966
data.loc[(data['income_type'] == 'пенсионер') & (data['total_income'].isna()),'total_income'] = 118514.486412
data.loc[(data['income_type'] == 'предприниматель') & (data['total_income'].isna()),'total_income'] = 499163.144947
data.loc[(data['income_type'] == 'сотрудник') & (data['total_income'].isna()),'total_income'] = 142594.396847
data.loc[(data['income_type'] == 'студент') & (data['total_income'].isna()),'total_income'] = 98201.625314

In [6]:
data['total_income'].isna().sum()

0

In [8]:
data['children'].value_counts()#не понятны данные по -1 и 20 детей. 

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [9]:
data['family_status'].value_counts() #все в норме

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [10]:
data['debt'].value_counts() #все в норме

0    19784
1     1741
Name: debt, dtype: int64

In [11]:
data['purpose'].value_counts() #все в норме

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

**Вывод**

Пропущенные значения в зарплате заменены на медиану для разных групп занятости

### Замена типа данных

In [12]:
# Данные в доходе приведу к типу int для удобства
data['total_income'] = data['total_income'].astype('int')
data['total_income'].value_counts()

142594    1105
172357     509
118514     414
150447     147
144533       3
          ... 
179138       1
61724        1
99269        1
151371       1
126820       1
Name: total_income, Length: 18608, dtype: int64

**Вывод**

Изменил тип данных в доходе на int для удобства работы с ними

### Обработка дубликатов

In [13]:
#Привел столбец образование к нижнему регистру
data['education'] = data['education'].str.lower()

In [14]:
data.duplicated().sum() 

71

In [15]:
data.drop_duplicates().reset_index(drop=True)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21450,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21451,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21452,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


**Вывод**

Дубликатов меньше 1%, удаление на статистиу не должно повлиять. Удалил 71 дубликата методом drop

### Лемматизация

In [16]:
m = Mystem()
lem = []
for i in data['purpose'].unique():
    lemmas =m.lemmatize(i)
    lem.append(lemmas)
lem

[['покупка', ' ', 'жилье', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['дополнительный', ' ', 'образование', '\n'],
 ['сыграть', ' ', 'свадьба', '\n'],
 ['операция', ' ', 'с', ' ', 'жилье', '\n'],
 ['образование', '\n'],
 ['на', ' ', 'проведение', ' ', 'свадьба', '\n'],
 ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'],
 ['покупка', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'],
 ['недвижимость', '\n'],
 ['строительство', ' ', 'недвижимость', '\n'],
 ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'],
 ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'],
 ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['жилье', '\n'],
 ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'],
 ['автомобиль'

**Вывод**

Провел леммитизацию данных столбца "цель получения кредита". На этой основе буду добавлять общий столбец цели кредита.

### Категоризация данных

In [17]:
# добавлю столбец с обобщенными целями кредита
m = Mystem()
def group_purpose(purpose):
    purpose = m.lemmatize(purpose) 
    if 'свадьба' in purpose:
        return 'свадьба'
    elif 'недвижимость' in purpose or 'жилье' in purpose:
        return 'недвижимость'
    elif 'автомобиль' in purpose:
        return 'автомобиль'
    elif 'образование' in purpose:
        return 'образование'

data['group_purpose'] = data['purpose'].apply(group_purpose)

data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,group_purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


In [18]:
data['group_purpose'].isna().sum() # пропусков нет

0

In [19]:
# добавлю столбец по количеству детей
data['children_group'] = data['children']

def children_group(children):
    if children == 0:
        return 'нет детей'
    else:
        return 'есть дети'
data['children_group'] = data['children_group'].apply(children_group)
data['children_group'].value_counts()

нет детей    14149
есть дети     7376
Name: children_group, dtype: int64

In [20]:
# добавил столбец'debt_status' со строковым значение по задолженности
data['debt_status'] = data['debt']
def debt_status(debt):
    if debt == 0:
        return 'нет задолженности'
    elif debt == 1:
        return 'есть задолженность'
data['debt_status'] = data['debt_status'].apply(debt_status)
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,group_purpose,children_group,debt_status
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,есть дети,нет задолженности
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,есть дети,нет задолженности
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,нет детей,нет задолженности
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,есть дети,нет задолженности
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,нет детей,нет задолженности


In [21]:
data['debt_status'].isna().sum() # пропусков нет

0

In [22]:
# найду мин, макс, медиану и средний доход и на основе их сделаю обобщенный столбец по доходам
display(data['total_income'].max())
display(data['total_income'].mean())
display(data['total_income'].median())
data['total_income'].min()

2265604

165224.81602787456

142594.0

20667

In [23]:
data['group_income'] = data['total_income']

def group_income(total):
    if total <= 50000:
        return 'низкий доход'
    elif total <= 165158:
        return 'ниже среднего'
    elif total <= 500000:
        return 'выше среднего'
    elif total > 500000:
        return 'высокий доход'

data['group_income'] = data['group_income'].apply(group_income)    
data['group_income'].value_counts()

ниже среднего    12953
выше среднего     7978
низкий доход       372
высокий доход      222
Name: group_income, dtype: int64

In [24]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,group_purpose,children_group,debt_status,group_income
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,есть дети,нет задолженности,выше среднего
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,есть дети,нет задолженности,ниже среднего
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,нет детей,нет задолженности,ниже среднего
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,есть дети,нет задолженности,выше среднего
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,нет детей,нет задолженности,ниже среднего


In [25]:
# для удобства работы дальше создам таблицу только с нужными мне столбцами
data_filter = data.loc[:, ['family_status', 'group_purpose', 'children_group', 'group_income', 'debt_status', 'debt']]
data_filter.head(5)

,family_status,group_purpose,children_group,group_income,debt_status,debt
0,женат / замужем,недвижимость,есть дети,выше среднего,нет задолженности,0
1,женат / замужем,автомобиль,есть дети,ниже среднего,нет задолженности,0
2,женат / замужем,недвижимость,нет детей,ниже среднего,нет задолженности,0
3,женат / замужем,образование,есть дети,выше среднего,нет задолженности,0
4,гражданский брак,свадьба,нет детей,ниже среднего,нет задолженности,0


**Вывод**

Провел классификацию по цели приоберетения кредита, уровню дохода, количества детей. Добавил столбец 'debt_status' о задолженности. 

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [26]:
pivot_children = data_filter.pivot_table(index='children_group', columns='debt_status',values='debt', aggfunc='count')
pivot_children['процент не возврата'] = pivot_children['есть задолженность'] / pivot_children['нет задолженности'] * 100
pivot_children['процент не возврата'] = pivot_children['процент не возврата'].astype('int')
pivot_children.sort_values(by='процент не возврата', ascending=False)

debt_status,есть задолженность,нет задолженности,процент не возврата
children_group,,,
есть дети,678,6698,10
нет детей,1063,13086,8


**Вывод**

Клиенты без детей самые надежные

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [27]:
pivot_family = data_filter.pivot_table(index='family_status', columns='debt_status',values='debt', aggfunc='count')
pivot_family['процент не возврата'] = pivot_family['есть задолженность'] / pivot_family['нет задолженности'] * 100
pivot_family['процент не возврата'] = pivot_family['процент не возврата'].astype('int')
pivot_family.sort_values(by='процент не возврата', ascending=False)

debt_status,есть задолженность,нет задолженности,процент не возврата
family_status,,,
Не женат / не замужем,274,2539,10
гражданский брак,388,3789,10
женат / замужем,931,11449,8
в разводе,85,1110,7
вдовец / вдова,63,897,7


**Вывод**

Клиенты со статусом "не женат/не замужем" и "гражданский брак" чаще становятся должниками

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [28]:
pivot_income = data_filter.pivot_table(index='group_income', columns='debt_status',values='debt', aggfunc='count')
pivot_income['процент не возврата'] = pivot_income['есть задолженность'] / pivot_income['нет задолженности'] * 100
pivot_income['процент не возврата'] = pivot_income['процент не возврата'].astype('int')
pivot_income.sort_values(by='процент не возврата', ascending=False)

debt_status,есть задолженность,нет задолженности,процент не возврата
group_income,,,
ниже среднего,1091,11862,9
выше среднего,613,7365,8
высокий доход,14,208,6
низкий доход,23,349,6


**Вывод**

Клиенты с низким и высоким доходом самые надежные

- Как разные цели кредита влияют на его возврат в срок?

In [29]:
pivot_purpose = data_filter.pivot_table(index='group_purpose', columns='debt_status',values='debt', aggfunc='count')
pivot_purpose['процент не возврата'] = pivot_purpose['есть задолженность'] / pivot_purpose['нет задолженности'] * 100
pivot_purpose['процент не возврата'] = pivot_purpose['процент не возврата'].astype('int')
pivot_purpose.sort_values(by='процент не возврата', ascending=False)

debt_status,есть задолженность,нет задолженности,процент не возврата
group_purpose,,,
автомобиль,403,3912,10
образование,370,3652,10
свадьба,186,2162,8
недвижимость,782,10058,7


**Вывод**

Клиенты с целью взять кредит на "автомобиль" и "образование" чаще становятся должниками

## Шаг 4. Общий вывод

Есть ли зависимость между наличием детей и возвратом кредита в срок?

    - Клиенты без детей самые надежные
Есть ли зависимость между семейным положением и возвратом кредита в срок?

    - Клиенты со статусом "не женат/не замужем" и "гражданский брак" чаще становятся должниками
Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

    - Клиенты с низким и высоким доходом самые надежные
Как разные цели кредита влияют на его возврат в срок?

    - Клиенты с целью взять кредит на "автомобиль" и "образование" чаще становятся должниками